## Set-up environment

As usual, we first install HuggingFace Transformers, and Datasets.

In [ ]:
!pip install pytorch_lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
import os
from pytorch_lightning.loggers import TensorBoardLogger
%load_ext autoreload
%autoreload 2

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Nov 17 05:45:55 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   28C    P0    44W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -q git+https://github.com/huggingface/transformers.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


In [ ]:
!pip install -q datasets

## Prepare data

Here we take a small portion of the IMDB dataset, a binary text classification dataset ("is a movie review positive or negative?").

In [ ]:
from datasets import load_dataset

train_ds, test_ds = load_dataset("imdb", split=['train', 'test[:6250]+test[-6250:]'])
train_ds, val_ds = load_dataset("imdb", split=['train', 'test[6250:12500]+test[-12500:-6250]'])

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:

print(test_ds)
print(val_ds)

Dataset({
    features: ['text', 'label'],
    num_rows: 12500
})
Dataset({
    features: ['text', 'label'],
    num_rows: 12500
})


We create id2label and label2id mappings, which are handy at inference time.

In [ ]:
labels = train_ds.features['label'].names
print(labels)

['neg', 'pos']


In [ ]:
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}
print(id2label)

{0: 'neg', 1: 'pos'}


Next, we prepare the data for the model using the tokenizer. 

In [ ]:
from transformers import PerceiverTokenizer

tokenizer = PerceiverTokenizer.from_pretrained("deepmind/language-perceiver")

train_ds = train_ds.map(lambda examples: tokenizer(examples['text'], padding="max_length", truncation=True),
                        batched=True)
val_ds = val_ds.map(lambda examples: tokenizer(examples['text'], padding="max_length", truncation=True),
                      batched=True)
test_ds = test_ds.map(lambda examples: tokenizer(examples['text'], padding="max_length", truncation=True),
                      batched=True)

Downloading:   0%|          | 0.00/668 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/879 [00:00<?, ?B/s]

Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.


  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/13 [00:00<?, ?ba/s]

  0%|          | 0/13 [00:00<?, ?ba/s]

We set the format to PyTorch tensors, and create familiar PyTorch dataloaders.

In [ ]:
train_ds.set_format(type="torch", columns=['input_ids', 'label'])
test_ds.set_format(type="torch", columns=['input_ids', 'label'])
val_ds.set_format(type="torch", columns=['input_ids', 'label'])

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_ds, batch_size=100, shuffle=True)
test_dataloader = DataLoader(test_ds, batch_size=25)
val_dataloader = DataLoader(val_ds, batch_size=25)

Here we verify some things (always important to check out your data!).

In [ ]:
batch = next(iter(train_dataloader))
for k,v in batch.items():
  print(k,v.shape)

label torch.Size([100])
input_ids torch.Size([100, 2048])


In [ ]:
tokenizer.decode(batch['input_ids'][3])

"[CLS]This film would usually classify as the worst movie production ever. Ever. But in my opinion it is possibly the funniest. The horrifying direction and screenplay makes this film priceless. I bought the movie whilst sifting through the bargain DVD's at my local pound shop. Me and some friends then watched it, admittedly whilst rather drunk. It soon occurred that this wasn't any normal film. Instead a priceless relic of what will probably be James Cahill's last film. At first we were confused and were screaming for the DVD player to be turned off but thankfully in our abnormal state no-one could be bothered. Instead we watched the film right through. At the end we soon realised we had found any wasters dream, something that you can acceptably laugh at for hours, whilst laughing for all the wrong reasons. We soon showed all our other friends and they too agreed, this wasn't a work of abysmal film. This was a film that you can truly wet yourself laughing at. This was a film that anyo

In [ ]:
# batch['label']

In [ ]:
import numpy as np
train_ds['label'].double().mean()
print(train_ds['label'][12499])

tensor(0)


## Define model

Next, we define our model, and put it on the GPU.

In [ ]:
# preprocessor we customized to use the tagkop encoder
# Must include tagkop_encoding_functions.py and pos_embeddings_IMDB_tSNE_2048x64.pth in base directory
from tagkop_encoding_functions import (
    PerceiverImagePreprocessor,
    TagkopPerceiverTextPreprocessor,
)
from transformers import PerceiverForSequenceClassification

import torch

from transformers.models.perceiver.modeling_perceiver import (
    PerceiverConfig,
    PerceiverModel,
    PerceiverClassificationDecoder,
    PerceiverTextPreprocessor,
    PerceiverClassificationDecoder
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


config = PerceiverConfig(
    num_self_attends_per_block = 4,
    d_model = 64
)

print('config', config)

MAE-Derived Perceiver Encodings
preprocessor = TagkopPerceiverTextPreprocessor(config)


decoder = PerceiverClassificationDecoder(config,
                                          num_channels=config.d_latents,
                                          trainable_position_encoding_kwargs=dict(num_channels=config.d_latents, index_dims=1),
                                          use_query_residual=True,
                                         )

model = PerceiverModel(config, input_preprocessor=preprocessor, decoder=decoder)



model.to(device)

ModuleNotFoundError: ignored

In [ ]:
# you can then do a forward pass as follows:
tokenizer = PerceiverTokenizer()
text = "hello world"
inputs = tokenizer(text, return_tensors="pt").input_ids
print(inputs)
inputs.to(device)
with torch.no_grad():
   outputs = model(inputs=inputs.to(device))
logits = outputs.logits
print('list(logits.shape): ', list(logits.shape))
# to train, one can train the model using standard cross-entropy:
criterion = torch.nn.CrossEntropyLoss()
labels = torch.tensor([1]).to(device)
loss = criterion(logits, labels)

## Train the model

Here we train the model using native PyTorch.

In [ ]:
from transformers import AdamW
from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score

from tqdm.notebook import tqdm
from datasets import load_metric

optimizer = AdamW(model.parameters(), lr=1e-4)

model.train()

batch = next(iter(train_dataloader))
for epoch in range(50):  # loop over the dataset multiple times

    epoch_accs = np.array([])
    # Add model save location below
    # torch.save(model.state_dict(), '/content/drive/MyDrive/saved_model/small_model_tagop_Nov_16_epoch_' + str(epoch) + '.pt')
    print('saved model')
    print("Epoch:", epoch)
    # for i in range(2):
    for batch in tqdm(train_dataloader):
         batch_accs = np.array([])
         # get the inputs; 
         inputs = batch["input_ids"].to(device)
        #  attention_mask = batch["attention_mask"].to(device)
         labels = batch["label"].to(device)

         # zero the parameter gradients
         optimizer.zero_grad()

         # forward + backward + optimize
         outputs = model(inputs=inputs)
         logits = outputs.logits
         
         # to train, one can train the model using standard cross-entropy:
         criterion = torch.nn.CrossEntropyLoss()

         loss = criterion(logits, labels)
         loss.backward()
         optimizer.step()
         
         # evaluate
         predictions = outputs.logits.argmax(-1).cpu().detach().numpy()
         accuracy = accuracy_score(y_true=batch["label"].numpy(), y_pred=predictions)
         print(f"Loss: {loss.item()}, Accuracy: {accuracy}")
         batch_accs = np.append(batch_accs, accuracy)
    
    print('Batch Accuracy Avg: ', np.mean(batch_accs))
    
    accuracy = load_metric("accuracy")
    
    # Validation Test
    model.eval()
    for batch in tqdm(val_dataloader):
          # get the inputs; 
          inputs = batch["input_ids"].to(device)
          labels = batch["label"].to(device)

          # forward pass
          outputs = model(inputs=inputs)
          logits = outputs.logits 
          predictions = logits.argmax(-1).cpu().detach().numpy()
          references = batch["label"].numpy()
          accuracy.add_batch(predictions=predictions, references=references)

    final_score = accuracy.compute()
    print("Accuracy on validation set:", final_score)
    epoch_accs = np.append(epoch_accs, final_score)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


saved model
Epoch: 0


  0%|          | 0/250 [00:00<?, ?it/s]

Loss: 0.6997937560081482, Accuracy: 0.59
Loss: 5.860103130340576, Accuracy: 0.47
Loss: 1.0586258172988892, Accuracy: 0.49
Loss: 0.7263509631156921, Accuracy: 0.41
Loss: 1.219814658164978, Accuracy: 0.47
Loss: 0.7055288553237915, Accuracy: 0.54
Loss: 0.889110803604126, Accuracy: 0.56
Loss: 0.9297664165496826, Accuracy: 0.54
Loss: 0.7063116431236267, Accuracy: 0.51
Loss: 0.8001638650894165, Accuracy: 0.52
Loss: 0.8801435828208923, Accuracy: 0.47
Loss: 0.7252963781356812, Accuracy: 0.44
Loss: 0.8669946193695068, Accuracy: 0.4
Loss: 0.8065783977508545, Accuracy: 0.47
Loss: 0.7030810713768005, Accuracy: 0.44
Loss: 0.8251209259033203, Accuracy: 0.42
Loss: 0.832796037197113, Accuracy: 0.43
Loss: 0.701611340045929, Accuracy: 0.48
Loss: 0.68707674741745, Accuracy: 0.58
Loss: 0.7653590440750122, Accuracy: 0.56
Loss: 0.8590285778045654, Accuracy: 0.47
Loss: 0.7559733390808105, Accuracy: 0.41
Loss: 0.7004354596138, Accuracy: 0.56
Loss: 0.9003586769104004, Accuracy: 0.49
Loss: 0.9321032762527466, A

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:49: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate


...

Accuracy on validation set: {'accuracy': 0.60528}
saved model
Epoch: 49


  0%|          | 0/250 [00:00<?, ?it/s]

Loss: 0.2540372908115387, Accuracy: 0.87
Loss: 0.24622032046318054, Accuracy: 0.88
Loss: 0.20250073075294495, Accuracy: 0.93
Loss: 0.18723514676094055, Accuracy: 0.9
Loss: 0.19631098210811615, Accuracy: 0.89
Loss: 0.17129811644554138, Accuracy: 0.93
Loss: 0.20397424697875977, Accuracy: 0.93
Loss: 0.16012921929359436, Accuracy: 0.9
Loss: 0.24851404130458832, Accuracy: 0.88
Loss: 0.25202542543411255, Accuracy: 0.89
Loss: 0.1167372614145279, Accuracy: 0.95
Loss: 0.17603947222232819, Accuracy: 0.95
Loss: 0.16904594004154205, Accuracy: 0.95
Loss: 0.17749334871768951, Accuracy: 0.94
Loss: 0.203091099858284, Accuracy: 0.92
Loss: 0.14064723253250122, Accuracy: 0.97
Loss: 0.30937880277633667, Accuracy: 0.89
Loss: 0.24844114482402802, Accuracy: 0.88
Loss: 0.20715168118476868, Accuracy: 0.93
Loss: 0.13639488816261292, Accuracy: 0.97
Loss: 0.23486702144145966, Accuracy: 0.91
Loss: 0.19155223667621613, Accuracy: 0.94
Loss: 0.11080603301525116, Accuracy: 0.94
Loss: 0.17072126269340515, Accuracy: 0.9

  0%|          | 0/500 [00:00<?, ?it/s]

Accuracy on validation set: {'accuracy': 0.60992}


## Evaluate the model

Finally, we evaluate the model on the test set. We use the Datasets library to compute the accuracy.

In [ ]:
# torch.save(model.state_dict(), '/content/drive/MyDrive/saved_model/small_model_tagop_Nov_16_epoch_last.pt')

In [ ]:
# Load model weights if desired by uncommenting and using your own pre-trained weights
# import torch
# checkpoint = torch.load('/content/drive/MyDrive/saved_model/small_network_model.pt')
# model.load_state_dict(checkpoint)
# model.eval()

In [ ]:
from tqdm.notebook import tqdm
from datasets import load_metric

accuracy = load_metric("accuracy")

model.eval()
for batch in tqdm(test_dataloader):
      # get the inputs; 
      inputs = batch["input_ids"].to(device)
      # attention_mask = batch["attention_mask"].to(device)
      labels = batch["label"].to(device)

      # forward pass
      outputs = model(inputs=inputs)
      logits = outputs.logits 
      predictions = logits.argmax(-1).cpu().detach().numpy()
      references = batch["label"].numpy()
      accuracy.add_batch(predictions=predictions, references=references)

final_score = accuracy.compute()
print("Accuracy on test set:", final_score)

  0%|          | 0/500 [00:00<?, ?it/s]

Accuracy on test set: {'accuracy': 0.61232}


## Inference

In [ ]:
text = "I hated this movie, it's really bad."

input_ids = tokenizer(text, return_tensors="pt").input_ids

# forward pass
outputs = model(inputs=input_ids.to(device))
logits = outputs.logits 
predicted_class_idx = logits.argmax(-1).item()

print("Predicted:", model.config.id2label[predicted_class_idx])